<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
from laserembeddings import Laser

In [ ]:
laser = Laser()

In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)
farsi.head()

,id,text,ref1,ref2,link,norm_text,google,mbart,ref3
0,0,بچه های عزیز این کشتزار منو بلاک کرده یکی که د...,1.0,0.0,https://twitter.com/DrErovay/status/1249849840...,بچه‌های عزیز این کشتزار منو بلاک کرده یکی که د...,Children's Menu block the field of view of one...,"My dear children, I've been blocked on this fa...",1.0
1,1,سیاسیون محترم از نمایندگان گرفته تا شهردارا...,0.0,0.0,https://twitter.com/atefeh heidari/status/1007...,سیاسیون محترم از نمایندگان گرفته تا شهرداران ...,Distinguished delegates politicians to munici...,From respectable politicians to elected offici...,0.0
2,2,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,1.0,0.0,https://twitter.com/fariborz c/status/10124857...,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,I disagree with execution\n\nIf only way to an...,"With the execution of my opponent, I wish ther...",1.0
3,3,دونوع آدم داریم : یکی اونکه هیچی به کیرش نیست ...,1.0,0.0,https://twitter.com/llil ili lill/status/10190...,دونوع آدم داریم: یکی اونکه هیچی به کیرش نیست و...,There are two types of people: one Avnkh nothi...,There are two types of people: There's the one...,0.0
4,4,هيچ زنداني بدتر از ،دختر بودن، نيست....,1.0,0.0,https://twitter.com/virgoOoOol/status/10233572...,هیچ زندانی بدتر از، دختر بودن، نیست ….,"No prisoners worse, the girl is not ....",There is no prison worse than being a girl...,0.0


In [ ]:
eng.head()

,Unnamed: 0,tweet,class
0,0,"""@USER @USER @USER queer"" gaywad",1
1,1,"""@USER @USER alsarabsss"" hes a beaner smh you ...",1
2,2,"""@USER @USER you're fucking gay, blacklisted h...",1
3,3,"""@USER LMFAOOOO I HATE BLACK PEOPLE https://t....",1
4,4,"""@USER ""At least I'm not a nigger"" http://t.co...",1


In [ ]:
eng = eng.drop("Unnamed: 0", axis=1)

In [ ]:
farsi = farsi[['norm_text', 'ref3']]
farsi.columns = ['tweet', 'class']
farsi.head()

,tweet,class
0,بچه‌های عزیز این کشتزار منو بلاک کرده یکی که د...,1.0
1,سیاسیون محترم از نمایندگان گرفته تا شهرداران ...,0.0
2,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,1.0
3,دونوع آدم داریم: یکی اونکه هیچی به کیرش نیست و...,0.0
4,هیچ زندانی بدتر از، دختر بودن، نیست ….,0.0


In [ ]:
farsi.tail()

,tweet,class
5196,پسرا، هم جنسا، چرا دختر هرچی بددهن باشه و کصشع...,0.0
5197,هدایت و بازداشت شده است. \nامارات سیاست سکوت ر...,0.0
5198,عزیزان، کسی گفته کاربران توئیتر مستجاب الدعوه‌...,0.0
5199,میگن هیچ چیزی بدون دلیل نیست\nهمه چی دست به دس...,0.0
5200,NaN,NaN


In [ ]:
farsi = farsi.dropna()
farsi.tail()

,tweet,class
5195,من ایشون میشناسم تو اینستام پیج داره تا جایی ک...,0.0
5196,پسرا، هم جنسا، چرا دختر هرچی بددهن باشه و کصشع...,0.0
5197,هدایت و بازداشت شده است. \nامارات سیاست سکوت ر...,0.0
5198,عزیزان، کسی گفته کاربران توئیتر مستجاب الدعوه‌...,0.0
5199,میگن هیچ چیزی بدون دلیل نیست\nهمه چی دست به دس...,0.0


In [ ]:
!pip install hazm

In [ ]:

def deEmojify(row):
    text = row.tweet
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
eng['tweet'] = eng.apply(deEmojify, axis=1)

In [ ]:
eng = eng.sample(frac=1).reset_index(drop=True)

In [ ]:
eng.head()

,tweet,class
0,"Dear , ICC instead of banning MS Dhoni to wea...",0
1,"@USER hey James your music still sucks, anyone...",1
2,"Absolutely take it, Worst cricket world cup ev...",0
3,Changed in last five years 1.First decide ...,0
4,@USER Where are you? Why isn't this case with ...,1


In [ ]:
eng['embeddings'] = eng.apply(lambda row:laser.embed_sentences( [row.tweet]
    ,
    lang='en'), axis=1)


In [ ]:
eng['embeddings'] = eng.apply(lambda row:row.embeddings[0], axis=1)


In [ ]:
eng.head()

,tweet,class,embeddings
0,"Dear , ICC instead of banning MS Dhoni to wea...",0,"[0.0031174975, 0.0023718362, -0.0018391929, 0...."
1,"@USER hey James your music still sucks, anyone...",1,"[0.008926708, 0.010169069, -0.002842401, 0.008..."
2,"Absolutely take it, Worst cricket world cup ev...",0,"[0.012971832, -1.96477e-05, 0.040654447, 0.033..."
3,Changed in last five years 1.First decide ...,0,"[0.047831852, 0.010176154, 7.78924e-05, -0.000..."
4,@USER Where are you? Why isn't this case with ...,1,"[0.008555521, 0.0035703606, 0.0054964153, 0.01..."


In [ ]:
y = eng["class"]
x = eng[['tweet', 'embeddings']]

In [ ]:
# english
x_train =  pd.DataFrame(x['embeddings'].to_list())
y_train = y.astype(dtype='float32')

In [ ]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.003117,0.002372,-0.001839,0.022990,0.004446,0.052800,0.007595,0.023102,0.010597,0.008126,0.049275,0.006277,0.008518,0.000700,0.014200,0.014805,0.037084,0.010441,0.008030,0.031470,0.030859,0.063824,-0.000010,0.024079,0.017756,0.012099,0.011254,0.035956,0.001600,0.022081,0.045858,0.002794,0.059937,0.009157,0.000855,0.018354,0.024751,0.027238,-0.000836,-0.000001,...,0.025749,-0.000808,0.017026,-0.000052,0.051626,0.055887,0.040003,0.000106,0.026241,0.023992,0.047635,-0.000550,0.002601,-0.002015,0.023069,0.002487,0.000756,0.027046,0.001640,0.044396,0.025927,0.051886,0.001350,0.021142,0.003638,0.001641,0.018996,-0.000539,0.003806,0.001092,0.035582,0.002544,0.003117,0.024228,0.003673,0.001072,0.020589,0.030277,0.031182,0.030853
1,0.008927,0.010169,-0.002842,0.008693,0.015077,0.011100,0.002808,0.041029,0.046282,-0.000234,0.007076,0.001384,0.000315,0.039544,0.008078,-0.000681,0.038611,0.002871,0.010935,0.005911,-0.002042,-0.000118,-0.000090,0.001145,0.037311,0.072575,0.024628,0.014092,0.002348,0.003903,0.056931,0.005750,0.002479,0.026939,-0.001038,0.014694,0.007875,0.007915,0.040112,0.000140,...,0.001305,-0.002465,0.014136,0.005926,-0.002175,0.005772,0.043182,0.001529,0.013578,0.013691,0.010144,-0.000626,-0.000263,0.011322,0.013324,-0.000251,0.000468,0.023615,0.011808,0.041421,0.013095,0.011158,0.014184,0.000969,0.007882,0.025693,0.024113,-0.001354,-0.001986,0.001421,0.036700,0.001560,0.008112,-0.002955,0.006415,0.005416,0.020413,0.016052,0.023731,0.025050
2,0.012972,-0.000020,0.040654,0.033012,0.017886,0.005245,0.000226,0.031476,0.018842,0.005589,0.059239,0.003095,0.000356,0.021311,0.026290,0.029194,0.009528,0.016297,0.000275,0.030084,0.015545,0.019726,-0.000049,0.015777,0.021716,0.000261,0.000061,0.007039,-0.000404,0.015720,0.002155,0.019679,0.004024,0.014902,0.018226,0.004216,0.031718,0.012778,0.000006,0.000047,...,0.027003,0.000609,0.009085,0.006057,0.035851,0.047674,0.037715,-0.000247,0.025907,0.025005,0.001000,-0.000435,-0.000186,0.026766,0.026106,0.005349,0.002157,-0.001236,0.013900,0.033635,0.031776,0.036311,0.048544,-0.000449,0.011325,0.005910,0.029098,-0.000529,0.003543,0.001510,0.053596,0.009492,0.006391,0.003279,0.016281,0.002777,0.017728,0.021583,0.010171,0.038417
3,0.047832,0.010176,0.000078,-0.000409,0.010386,0.007353,0.007708,0.047639,-0.002822,0.067938,0.015368,0.007629,0.007121,0.005021,0.004876,0.034954,0.040596,0.002970,0.007224,0.022856,0.051820,0.086115,-0.000019,0.027434,0.049204,0.048782,0.004330,0.006554,0.001175,0.018723,0.045369,0.017861,0.011729,0.012486,0.056494,0.033724,0.050707,0.025786,0.002289,0.000156,...,0.006713,0.001430,0.022090,0.003149,0.012095,0.016555,0.023127,0.007705,0.030804,0.025527,-0.000246,0.004194,0.003710,0.021119,0.013888,0.005726,-0.000323,0.010194,0.005931,0.037300,0.004974,0.020111,0.009727,-0.000054,0.001553,-0.001242,0.006333,0.000372,0.002442,0.000904,-0.003327,0.041326,0.004831,0.029180,0.018356,0.008074,0.024054,0.066603,0.012784,0.033870
4,0.008556,0.003570,0.005496,0.010051,0.007730,0.045267,0.049428,0.026842,0.012675,0.001223,0.028820,0.000712,0.004302,0.002887,0.005704,0.008727,0.034685,0.009686,0.019069,0.000137,0.021424,0.064962,0.000869,0.012280,0.029029,0.003381,0.011271,0.029281,-0.000650,0.005504,0.045089,0.026013,-0.000662,0.010328,0.001270,0.018917,0.024698,0.017924,0.000437,-0.000043,...,0.044671,-0.000055,0.019660,0.001630,-0.001932,0.042147,0.052855,-0.000019,0.030112,0.006085,0.001021,-0.000324,0.014961,0.005190,0.028613,0.001247,0.002776,0.004919,0.024057,0.038780,0.013592,0.018443,0.006194,0.001207,0.008388,-0.001365,0.029492,-0.000073,-0.000065,0.000731,0.050573,0.020767,0.000379,0.002085,0.011848,0.002977,0.018112,0.044516,0.026557,0.041415


In [ ]:
farsi['tweet'] = farsi.apply(deEmojify, axis=1)

In [ ]:
farsi = farsi.sample(frac=1).reset_index(drop=True)

In [ ]:
farsi.head()

,tweet,class
0,با این حال، شما کتاب «خداحافظ‌گری کوپر» را برد...,0.0
1,من تو این ٥ ماه متوجه شدم تمام بچه‌های اروپایی...,0.0
2,در کوردستان احزاب کوردستانی با مردم خود بیعت ت...,0.0
3,دختر آبی برای حقوق از دست رفته‌اش جنگید و به م...,0.0
4,بهترین موضع آمریکا در قبال توقیف نفتکش بریتانی...,1.0


In [ ]:
farsi['embeddings'] = farsi.apply(lambda row:laser.embed_sentences( [row.tweet]
    ,
    lang='pes'), axis=1)

In [ ]:
farsi.head()

,tweet,class,embeddings
0,با این حال، شما کتاب «خداحافظ‌گری کوپر» را برد...,0.0,"[[0.014829095, 0.020921092, 0.0014574535, 0.00..."
1,من تو این ٥ ماه متوجه شدم تمام بچه‌های اروپایی...,0.0,"[[0.016591031, 0.009440024, 0.0018926356, 0.01..."
2,در کوردستان احزاب کوردستانی با مردم خود بیعت ت...,0.0,"[[0.005744106, 0.0065343273, 0.0020629675, 0.0..."
3,دختر آبی برای حقوق از دست رفته‌اش جنگید و به م...,0.0,"[[0.012158961, -4.1281505e-06, 0.0028556427, 0..."
4,بهترین موضع آمریکا در قبال توقیف نفتکش بریتانی...,1.0,"[[0.008261064, 0.0026255003, 0.0041782763, 0.0..."


In [ ]:
farsi['embeddings'] = farsi.apply(lambda row:row.embeddings[0], axis=1)

In [ ]:
farsi.head()

,tweet,class,embeddings
0,با این حال، شما کتاب «خداحافظ‌گری کوپر» را برد...,0.0,"[0.014829095, 0.020921092, 0.0014574535, 0.006..."
1,من تو این ٥ ماه متوجه شدم تمام بچه‌های اروپایی...,0.0,"[0.016591031, 0.009440024, 0.0018926356, 0.011..."
2,در کوردستان احزاب کوردستانی با مردم خود بیعت ت...,0.0,"[0.005744106, 0.0065343273, 0.0020629675, 0.02..."
3,دختر آبی برای حقوق از دست رفته‌اش جنگید و به م...,0.0,"[0.012158961, -4.1281505e-06, 0.0028556427, 0...."
4,بهترین موضع آمریکا در قبال توقیف نفتکش بریتانی...,1.0,"[0.008261064, 0.0026255003, 0.0041782763, 0.01..."


In [ ]:
y_fa = farsi["class"]
x_fa = pd.DataFrame(farsi['embeddings'].to_list())

In [ ]:
x_tune, x_test, y_tune, y_test = train_test_split(x_fa, y_fa, test_size=0.4, random_state=101)

# New Section

In [ ]:
print(x_tune.shape, x_test.shape)

(3120, 1024) (2080, 1024)


In [ ]:
xx_train = x_train.values.tolist()
xx_test = x_test.values.tolist()
yy_train = y_train.values.astype(dtype='float32').tolist()
yy_test = y_test.values.astype(dtype='float32').tolist()
xx_tune = x_tune.values.tolist()
yy_tune = y_tune.values.tolist()

In [ ]:

import torch
cuda0 = torch.device('cuda:0')
xx_train = torch.tensor(xx_train, dtype=torch.float32, device=cuda0)
xx_test = torch.tensor(xx_test, dtype=torch.float32, device=cuda0)
yy_train = torch.tensor(yy_train, dtype=torch.float32, device=cuda0)
yy_test = torch.tensor(yy_test, dtype=torch.float32, device=cuda0)
xx_tune = torch.tensor(xx_tune, dtype=torch.float32, device=cuda0)
yy_tune = torch.tensor(yy_tune, dtype=torch.float32, device=cuda0)

In [ ]:
from torch import nn
torch.manual_seed(101)
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(1024, 768)
    self.fc2 = nn.Linear(768, 128)
    self.fc3 = nn.Linear(128, 1)
    self.relu = nn.LeakyReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    f1 = self.relu(self.fc1(x))
    f2 = self.relu(self.fc2(f1))
    output = self.fc3(f2)
    output = self.sigmoid(output)
    return output

In [ ]:
# we want to change network architeture change below 

In [ ]:
# from torch import nn
# torch.manual_seed(101)
# class Network(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.fc1 = nn.Linear(1024, 768)
#     self.fc2 = nn.Linear(768, 1)
#   # self.fc3 = nn.Linear(128, 1)
#     self.relu = nn.LeakyReLU()
#     self.sigmoid = nn.Sigmoid()

#   def forward(self, x):
#     f1 = self.relu(self.fc1(x))
#   # f2 = self.relu(self.fc2(f1))
#     output = self.fc2(f1)
#     output = self.sigmoid(output)
#     return output

In [ ]:
model = Network()
model.to('cuda:0')

Network(
  (fc1): Linear(in_features=1024, out_features=768, bias=True)
  (fc2): Linear(in_features=768, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
)

In [ ]:

# define the optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 1e-5) 
n_epochs = 100 # or whatever
batch_size = 16 # or whatever

In [ ]:
xx_train.size()[0]

10853

In [ ]:
model.train()
torch.manual_seed(101)
for epoch in range(n_epochs):

    # X is a torch Variable
    #  permutation = torch.randperm(X.size()[0])

    for i in range(0,xx_train.size()[0], batch_size):
        optimizer.zero_grad()

        # indices = permutation[i:i+batch_size]
        batch_x, batch_y = xx_train[i:i+batch_size], yy_train[i:i+batch_size]

        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        loss = criterion(outputs.squeeze(),batch_y)

        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
y_pred = model(xx_tune)
y_pred = y_pred.squeeze()
y_pred = torch.round(y_pred)

In [ ]:
print(classification_report([int(i) for i in yy_tune], [int(j) for j in y_pred]))

              precision    recall  f1-score   support

           0       0.91      0.73      0.81      2654
           1       0.27      0.57      0.37       466

    accuracy                           0.71      3120
   macro avg       0.59      0.65      0.59      3120
weighted avg       0.81      0.71      0.74      3120



In [ ]:
# train using fine tune


In [ ]:
model.train()

for epoch in range(30):

    # X is a torch Variable
    #  permutation = torch.randperm(X.size()[0])

    for i in range(0,xx_tune.size()[0], batch_size):
        optimizer.zero_grad()

        # indices = permutation[i:i+batch_size]
        batch_x, batch_y = xx_tune[i:i+batch_size], yy_tune[i:i+batch_size]

        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        loss = criterion(outputs.squeeze(),batch_y)

        loss.backward()
        optimizer.step()

In [ ]:
model.eval()
yy_pred = model(xx_test)
yy_pred = yy_pred.squeeze()
yy_pred = torch.round(yy_pred)

In [ ]:
print("result for testing against dataset")

result for testing against dataset


In [ ]:
print(classification_report([int(i) for i in yy_test], [int(j) for j in yy_pred]))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      1782
           1       0.61      0.34      0.44       298

    accuracy                           0.87      2080
   macro avg       0.75      0.65      0.68      2080
weighted avg       0.86      0.87      0.86      2080



In [ ]:
# lets see what happens if we dont use english dataset and only use farsi dataset

In [ ]:
model_farsi_only = Network()
model_farsi_only.to('cuda:0')

Network(
  (fc1): Linear(in_features=1024, out_features=768, bias=True)
  (fc2): Linear(in_features=768, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
)

In [ ]:
# define the optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(model_farsi_only.parameters(),
                  lr = 1e-5) 
n_epochs = 100 # or whatever
batch_size = 16 # or whatever

In [ ]:
model_farsi_only.train()
torch.manual_seed(101)
for epoch in range(n_epochs):

    # X is a torch Variable
    #  permutation = torch.randperm(X.size()[0])

    for i in range(0,xx_tune.size()[0], batch_size):
        optimizer.zero_grad()

        # indices = permutation[i:i+batch_size]
        batch_x, batch_y = xx_tune[i:i+batch_size], yy_tune[i:i+batch_size]

        # in case you wanted a semi-full example
        outputs = model_farsi_only.forward(batch_x)
        loss = criterion(outputs.squeeze(),batch_y)

        loss.backward()
        optimizer.step()

In [ ]:
model_farsi_only.eval()
yy_pred = model_farsi_only(xx_test)
yy_pred = yy_pred.squeeze()
yy_pred = torch.round(yy_pred)
print(classification_report([int(i) for i in yy_test], [int(j) for j in yy_pred]))

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1782
           1       0.67      0.36      0.47       298

    accuracy                           0.88      2080
   macro avg       0.79      0.66      0.70      2080
weighted avg       0.87      0.88      0.87      2080

